### Important

In [2]:
data_name = 'WN18RR_v1'
model_id = 'main_4'

In [3]:
#difine the names for saving
model_name = 'Model_' + model_id + '_' + data_name
ids_name = 'IDs_' + model_id + '_' + data_name

In [4]:
import librosa
import opensmile
import os
import sys
import numpy as np
import random
from collections import defaultdict
from copy import deepcopy
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import initializers
from tensorflow.keras.utils import plot_model

In [41]:
class LoadKG:
    
    def __init__(self):
        
        self.x = 'Hello'
        
    def load_train_data(self, data_path, one_hop, data, s_t_r, entity2id, id2entity,
                     relation2id, id2relation):
        
        data_ = set()
    
        ####load the train, valid and test set##########
        with open (data_path, 'r') as f:
            
            data_ini = f.readlines()
                        
            for i in range(len(data_ini)):
            
                x = data_ini[i].split()
                
                x_ = tuple(x)
                
                data_.add(x_)
        
        ####relation dict#################
        index = len(relation2id)
     
        for key in data_:
            
            if key[1] not in relation2id:
                
                relation = key[1]
                
                relation2id[relation] = index
                
                id2relation[index] = relation
                
                index += 1
                
                #the inverse relation
                iv_r = '_inverse_' + relation
                
                relation2id[iv_r] = index
                
                id2relation[index] = iv_r
                
                index += 1
        
        #get the id of the inverse relation, by above definition, initial relation has 
        #always even id, while inverse relation has always odd id.
        def inverse_r(r):
            
            if r % 2 == 0: #initial relation
                
                iv_r = r + 1
            
            else: #inverse relation
                
                iv_r = r - 1
            
            return(iv_r)
        
        ####entity dict###################
        index = len(entity2id)
        
        for key in data_:
            
            source, target = key[0], key[2]
            
            if source not in entity2id:
                                
                entity2id[source] = index
                
                id2entity[index] = source
                
                index += 1
            
            if target not in entity2id:
                
                entity2id[target] = index
                
                id2entity[index] = target
                
                index += 1
                
        #create the set of triples using id instead of string        
        for ele in data_:
            
            s = entity2id[ele[0]]
            
            r = relation2id[ele[1]]
            
            t = entity2id[ele[2]]
            
            if (s,r,t) not in data:
                
                data.add((s,r,t))
            
            s_t_r[(s,t)].add(r)
            
            if s not in one_hop:
                
                one_hop[s] = dict()
            
            if r not in one_hop[s]:
                
                one_hop[s][r] = set()
            
            one_hop[s][r].add(t)
            
            if t not in one_hop:
                
                one_hop[t] = dict()
            
            r_inv = inverse_r(r)
            
            s_t_r[(t,s)].add(r_inv)
            
            if r_inv not in one_hop[t]:
                
                one_hop[t][r_inv] = set()
            
            one_hop[t][r_inv].add(s)

In [37]:
class ObtainPathsByDynamicProgramming:

    def __init__(self, size_bd=50, threshold=500000):
                
        self.size_bd = size_bd #size bound limit the number of paths to a target entity t
        
        #number of times paths with specific length been performed for recursion
        self.threshold = threshold
        
    #obtain all the connected entitis from one source s, by any paths
    #each connected target entity with their shortest path length will be recorded
    def obtain_all_connected_entity(self, s, one_hop):

        neighbor = {s: 0}

        for r in one_hop[s]:

            for t in one_hop[s][r]:

                if t not in neighbor:

                    neighbor[t] = 1

        completed = False
        length = 1

        while not completed:

            temp = set()

            for e in neighbor:

                for r in one_hop[e]:

                    for t in one_hop[e][r]:

                        if t not in neighbor:

                            temp.add(t)

            if len(temp) == 0:

                completed = True

            else:

                length += 1

                for t in temp:

                    neighbor[t] = length

        return(neighbor)
        
    '''
    Given an entity s, the function will find the paths from s to other entities, using recursion.
    
    One may refer to LeetCode Problem 797 for details:
        https://leetcode.com/problems/all-paths-from-source-to-target/
    '''
    def obtain_paths(self, mode, s, t_input, lower_bd, upper_bd, one_hop):

        if type(lower_bd) != type(1) or lower_bd < 1:
            
            raise TypeError("!!! invalid lower bound setting, must >= 1 !!!")
            
        if type(upper_bd) != type(1) or upper_bd < 1:
            
            raise TypeError("!!! invalid upper bound setting, must >= 1 !!!")
            
        if lower_bd > upper_bd:
            
            raise TypeError("!!! lower bound must not exced upper bound !!!")
            
        if s not in one_hop:
            
            raise ValueError('!!! entity not in one_hop. Please work on existing entities')
        
        #here is the result dict. Its key is each entity t sharing paths from s
        #The value of each t is a set containing the paths from s to t
        #These paths can be either the direct connection r, or a multi-hop path
        res = defaultdict(set)
        
        #qualified_t contains the types of t we want to consider,
        #that is, what t will be added to the result set.
        qualified_t = set()
        
        #under this mode, we will only consider the direct neighbour of s
        if mode == 'direct_neighbour':
            
            all_connected_to_t = dict()
        
            for r in one_hop[s]:
            
                for t in one_hop[s][r]:
                
                    qualified_t.add(t)
            
            #update the all_connected_to_t dict
            for t in qualified_t:

                temp_connected = self.obtain_all_connected_entity(t, one_hop)

                for key in temp_connected:

                    if key not in all_connected_to_t:

                        all_connected_to_t[key] = deepcopy(temp_connected[key])

                    else:

                        all_connected_to_t[key] = min(all_connected_to_t[key], temp_connected[key])
        
        #under this mode, we will only consider one specified entity t
        elif mode == 'target_specified':
            
            qualified_t.add(t_input)
            
            #all the connected entities from s to t
            all_connected_to_t = self.obtain_all_connected_entity(t_input, one_hop)
        
        #under this mode, we will consider any entity
        elif mode == 'any_target':
            
            #we want it to be empty
            all_connected_to_t = dict()
            
            for s_any in one_hop:
                
                qualified_t.add(s_any)
                
        else:
            
            raise ValueError('not a valid mode')
        
        '''
        We use recursion to find the paths
        On current node with the path [r1, ..., rk] and on-path entities {s, e1, ..., ek-1, node}
        from s to this node, we will further find the direct neighbor t' of this node. 
        If t' is not an on-path entity (not among s, e1,...ek-1, node), we recursively proceed to t' 
        '''
        def helper(node, path, on_path_en, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict):

            #when the current path is within lower_bd and upper_bd, 
            #and the node is among the qualified t, and it has not been fill of paths w.r.t size_limit,
            #we will add this path to the node
            if (len(path) >= lower_bd) and (len(path) <= upper_bd) and (
                node in qualified_t) and (len(res[node]) < self.size_bd):
                
                res[node].add(tuple(path))
                    
            #won't start new recursions if the current path length already reaches upper limit, or
            #the number of recursion on this length has reached the limit or
            if (len(path) < upper_bd) and (count_dict[len(path)] <= self.threshold):
         
                #find all the directly connected (r,t) pair to this current node
                #these pairs indicates potentially the next recursion
                potential_pair = set()

                for r in one_hop[node]:

                    for t in one_hop[node][r]:

                        potential_pair.add((r,t))

                '''here, we use the entities connected to t as priority'''
                #the more the entity close to t, the higher it will be ranked to recurse ahead
                priority_list = list()
                
                for Tuple in potential_pair:
                    
                    r, t = Tuple[0], Tuple[1]
                    
                    if t in all_connected_to_t:
                        
                        priority_list.append([Tuple, min(all_connected_to_t[t], upper_bd)])
                        
                    else:
                        
                        priority_list.append([Tuple, upper_bd])
                        
                priority_list.sort(key = lambda x: x[-1])
                
                for List in priority_list:
                    
                    r, t = List[0][0], List[0][1]
                    
                    count_dict[len(path)] += 1
                    
                    #if t not on the path, then finally proceed to next recursion
                    if t not in on_path_en:

                        helper(t, path + [r], on_path_en.union({t}), res, qualified_t, 
                               lower_bd, upper_bd, one_hop, count_dict)
        
        length_dict = defaultdict(int)
        count_dict = defaultdict(int)
        
        helper(s, [], {s}, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict)
        
        return(res, count_dict)

In [43]:
train_path = '../data/' + data_name + '/train.txt'

In [44]:
#load the classes
Class_1 = LoadKG()
Class_2 = ObtainPathsByDynamicProgramming()

In [45]:
#define the dictionaries and sets for load KG
one_hop = dict() 
data = set()
s_t_r = defaultdict(set)
entity2id = dict()
id2entity = dict()
relation2id = dict()
id2relation = dict()

#fill in the sets and dicts
Class_1.load_train_data(train_path, one_hop, data, s_t_r,
                        entity2id, id2entity, relation2id, id2relation)

### Build the deep neural network structure

We use biLSTM to train on the input path embedding sequence to predict the output embedding or the relation.

In [9]:
# Input layer, using integer to represent each relation type
#note that inputs_path is the path inputs, while inputs_out_re is the output relation inputs
fst_path = keras.Input(shape=(None,), dtype="int32")
scd_path = keras.Input(shape=(None,), dtype="int32")

#the relation input layer (for output embedding)
id_rela = keras.Input(shape=(None,), dtype="int32")

# Embed each integer in a 300-dimensional vector as input,
# note that we add another "space holder" embedding, 
# which hold the spaces if the initial length of two paths are not the same
in_embd_var = layers.Embedding(len(relation2id)+1, 300)

# Obtain the embedding
fst_p_embd = in_embd_var(fst_path)
scd_p_embd = in_embd_var(scd_path)

# Embed each integer in a 300-dimensional vector as output
rela_embd = layers.Embedding(len(relation2id)+1, 300)(id_rela)

#add 2 layer bi-directional LSTM
lstm_layer_1 = layers.Bidirectional(layers.LSTM(150, return_sequences=True))
lstm_layer_2 = layers.Bidirectional(layers.LSTM(150, return_sequences=True))

#first LSTM layer
fst_lstm_mid = lstm_layer_1(fst_p_embd)
scd_lstm_mid = lstm_layer_1(scd_p_embd)

#second LSTM layer
fst_lstm_out = lstm_layer_2(fst_lstm_mid)
scd_lstm_out = lstm_layer_2(scd_lstm_mid)

###########################################
####apply the attention mechanism##########
#first expand the dimention at the end to get ready for conv2D: (Batch,time,300,1)
fst_exp_dim = tf.expand_dims(fst_lstm_out, axis=-1)
scd_exp_dim = tf.expand_dims(scd_lstm_out, axis=-1)

#define the attention layer using convolutional 2D: output is
att_layer_conv2D = layers.Conv2D(100, (1, 300), padding='valid', activation='relu', 
                   input_shape=(None, 300), data_format='channels_last')

#shape: (Batch,Time,1,100)
fst_att_mid = att_layer_conv2D(fst_exp_dim)
scd_att_mid = att_layer_conv2D(scd_exp_dim)

#squeeze out the dim 1 to become: (Batch, Time, 100)
fst_squez = tf.squeeze(fst_att_mid, 2)
scd_squez = tf.squeeze(scd_att_mid, 2)

#expand the dimention again to become (Batch, Time, 100, 1)
fst_exp_dim_2 = tf.expand_dims(fst_squez, axis=-1)
scd_exp_dim_2 = tf.expand_dims(scd_squez, axis=-1)

#obtain the attention score for each time step by another conv2D layer
att_layer_conv2D_2 = layers.Conv2D(1, (1, 100), padding='valid', activation='relu', 
                     input_shape=(None, 100), data_format='channels_last')

#obtain (Batch, Time, 1, 1)
fst_mid_score = att_layer_conv2D_2(fst_exp_dim_2)
scd_mid_score = att_layer_conv2D_2(scd_exp_dim_2)

#squeeze again to obtain (Batch, Time, 1)
fst_squez_2 = tf.squeeze(fst_mid_score, -1)
scd_squez_2 = tf.squeeze(scd_mid_score, -1)

#softmax the attention score
softmax_l = layers.Softmax(1) #define softmax

fst_att_score = softmax_l(fst_squez_2)
scd_att_score = softmax_l(scd_squez_2)

#multiply the attention score to lstm output
fst_att_befsum = layers.Multiply()([fst_lstm_out, fst_att_score])
scd_att_befsum = layers.Multiply()([scd_lstm_out, scd_att_score])

#sum over time dimension to complete the attention: (Batch, 300)
fst_att_out = tf.reduce_sum(fst_att_befsum, axis=1)
scd_att_out = tf.reduce_sum(scd_att_befsum, axis=1)
######################################

#concatenate the output vector from both siamese tunnel: (Batch, 600)
path_concat = layers.concatenate([fst_att_out, scd_att_out], axis=-1)

#multiply into output embd size by dense layer: (Batch, 300)
path_out_vect = layers.Dense(300, activation='tanh')(path_concat)

#remove the time dimension from the output embd since there is only one step
rela_out_embd = tf.reduce_sum(rela_embd, axis=1)

# Normalize the vectors to have unit length
path_out_vect_norm = tf.math.l2_normalize(path_out_vect, axis=-1)
rela_out_embd_norm = tf.math.l2_normalize(rela_out_embd, axis=-1)

# Calculate the dot product
dot_product = layers.Dot(axes=-1)([path_out_vect_norm, rela_out_embd_norm])

#put together the model
model = keras.Model([fst_path, scd_path, id_rela], dot_product)

2023-02-01 12:33:46.613953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
#config the Adam optimizer 
opt = keras.optimizers.Adam(learning_rate=0.0005, decay=1e-6)

#compile the model
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])

### Build the batches
We build each big-batch for each path combination with length (i,j). Then, we iteratively train the siamese network on different big-batches. The length of each big-batch is N.

To be specific:
* If we allow the length difference between two paths in a combination to be d, then the combination with path length i and path length j, denoted as (i,j), will be like (2,2), (2,3), (2,4), (3,3), (3,4), (3,5), ... 
* We will first build all the big-batches before fitting the NN model. 
* That is, we will perform the ObtainPathsByDynamicProgramming class function for some randomly chosen source entities. Then, for each target entity, we will further have two for loops:
* for path_1 in all the 
* Do this until all the slots in all big-batchs are filled.
* In every epoch, big-batchs will be re-filled.

Then, in the training, we will use negative sampling: In each batch (actual batch, not the big-batch), we will include K true output relation embeddings and K random selected output relation embeddings. The true label is [1,0], while the false label is [0,1].

In [46]:
#function to build all the big batches
def build_big_batches(holder_len, lower_bd, upper_bd, Class_2, one_hop, s_t_r,
                      x_p_list, x_r_list, y_list,
                      relation2id, entity2id, id2relation, id2entity):
    
    if holder_len % 10 != 0:
        raise ValueError('We would like to take 10X as a big-batch size')
    
    #the set of all relation IDs
    relation_id_set = set()
    for i in range(len(id2relation)):
        
        if i not in id2relation:
            raise ValueError('error when generaing id2relation')
        
        relation_id_set.add(i)
    
    num_r = len(id2relation)
    
    #count how many appending has performed
    count = 0

    #in case not all entities in entity2id are in one_hop, 
    #so we need to find out who are indeed in
    existing_ids = set()
    
    for s_1 in one_hop:
        existing_ids.add(s_1)
        
    existing_ids = list(existing_ids)
    
    carry_on = True
    
    while carry_on:

        #obtain paths by dynamic programming
        source_id = random.choice(existing_ids)

        result = Class_2.obtain_paths('direct_neighbour', source_id, 
                                                   'not_specified', lower_bd, upper_bd, one_hop)
        
        #We want to increase the diversity of paths and targets.
        #So we abandon one sub-graph from a source_id, if we sampled more than K1 path pairs
        #Note that we mean "sampled", not "appended"! 
        #We do not care whether the pair is actually appended.
        threshold_0 = 1000
        count_0 = 0
        
        for target_id in result:

            if (not carry_on) or (count_0 > threshold_0):
                break
            
            #we want to make sure s, t are indeed directly connected, 
            #otherwise there is no relation for positive sample
            #also, we want to make sure s and t and not connected by all relations, 
            #although this situation is rare. 
            #But in that case, there is no relation for negative samples
            #Also, we want at least two different paths here between s and t
            if ((source_id, target_id) in s_t_r) and (
                len(s_t_r[(source_id, target_id)]) < len(id2relation)) and (
                len(result[target_id]) >= 2):
                
                dir_r = list(s_t_r[(source_id, target_id)])
                
                non_dir_r = list(relation_id_set.difference(dir_r))
                
                if len(dir_r) <= 0:
                    
                    raise ValueError('errors when creating s_t_r !!')
                    
                temp_path_list = list(result[target_id])
                    
                #futhermore, we will abandon one targed_id if we sampled more than K2 times
                threshold_1 = 50
                count_1 = 0
                
                while count_1 <= threshold_1 and count_0 <= threshold_0:
                
                    temp_pair = random.sample(temp_path_list,2)
                    
                    path_1, path_2 = temp_pair[0], temp_pair[1]

                    #decide which path is shorter and which is longer
                    if len(path_1) <= len(path_2):

                        path_s, path_l = path_1, path_2

                    else:

                        path_s, path_l = path_2, path_1                            

                    if (len(path_s) < lower_bd) or (len(path_l) > upper_bd):

                        raise ValueError('something wrong with the path finding')

                    #proceed when the entire length not yet reached,
                    #and whether this path pair is new, and whether the two paths are different
                    #But it is optional to require the path to be new. 
                    #We may remove this requirment, especially for short paths
                    '''remember to cancel the comment below when using path_comb'''
                    if (carry_on) and (path_s != path_l):

                        #we always add one positive and one negative situation together,
                        #hence, the length of list should always be even.
                        #also we want to make sure the length of lists coincide
                        if (len(x_p_list['s']) != len(y_list)) or (
                            len(x_p_list['s']) != len(x_p_list['l'])) or (
                            len(y_list) != len(x_r_list)) or (
                            len(y_list) % 2 != 0):

                            raise ValueError('error when building big batches: length error')

                        #####positive#####################
                        #we randomly choose one direction relation as the target relation
                        relation_id = random.choice(dir_r)

                        #append the paths: note that we add the space holder id at the end
                        #of the shorter path
                        x_p_list['s'].append(list(path_s) + [num_r]*abs(len(path_s)-upper_bd))
                        x_p_list['l'].append(list(path_l) + [num_r]*abs(len(path_l)-upper_bd))

                        #append relation
                        x_r_list.append([relation_id])
                        y_list.append(1.)

                        #####negative#####################
                        relation_id = random.choice(non_dir_r)

                        #append the paths: note that we add the space holder id at the end
                        #of the shorter path
                        x_p_list['s'].append(list(path_s) + [num_r]*abs(len(path_s)-upper_bd))
                        x_p_list['l'].append(list(path_l) + [num_r]*abs(len(path_l)-upper_bd))

                        #append relation
                        x_r_list.append([relation_id])
                        y_list.append(0.)

                        ######add to path combinations#####
                        #here is the tricky part: we have to add both (path_s, path_l)
                        #and (path_l, path_s). This is because when the length are the same
                        #adding only one situation won't guarantee that 
                        #the same path with different order is also considered.
                        #in other words: path combination don't have order, but our dict does.
                        #so we have to add both situations.
                        '''remember to cancel the comment here when using path_comb'''
                        #path_comb[(len(path_s), len(path_l))].add((path_s, path_l))
                        #path_comb[(len(path_s), len(path_l))].add((path_l, path_s))

                        count += 2

                        if count % 20000 == 0:
                            print('generating big-batches', count, holder_len)

                    if len(y_list) >= holder_len:

                        carry_on = False
                        
                    count_1 += 1
                    count_0 += 1

### Start Training: load the KG and call classes

Here, we use the validation set to see the training efficiency. That is, we use the validation to check whether the true relation between entities can be predicted by paths.

The trick is: in validation, we have to use the same relation ID and entity ID as in the training. But we don't want to use the links in training anymore. That is, in validation, we want to use (and update if necessary) entity2id, id2entity, relation2id and id2relation. But we want to use new one_hop, data, data_ and s_t_r for validation set. Then, path-finding will also be based on new one_hop.


In [12]:
model_name

'Model_main_4_WN18RR_v1'

In [13]:
ids_name

'IDs_main_4_WN18RR_v1'

In [14]:
#first, we save the relation and ids
Dict = dict()
Dict['one_hop'] = one_hop
Dict['data'] = data
Dict['s_t_r'] = s_t_r
Dict['entity2id'] = entity2id
Dict['id2entity'] = id2entity
Dict['relation2id'] = relation2id
Dict['id2relation'] = id2relation

with open('../weight_bin/' + ids_name + '.pickle', 'wb') as handle:
    pickle.dump(Dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
holder_len = 400000
lower_bd = 2
upper_bd = 10
num_epoch = 10
batch_size = 4

#90% to be train, 10% to be validation
train_len = 9*int(holder_len/10)
    
######################################
###pre-define the lists###############

#define the lists
x_p_list, x_r_list, y_list = {'s': [], 'l': []}, list(), list()

#######################################
###build the big-batches###############      

#fill in the training array list
build_big_batches(holder_len, lower_bd, upper_bd, Class_2, one_hop, s_t_r,
                      x_p_list, x_r_list, y_list,
                      relation2id, entity2id, id2relation, id2entity)

#######################################
###do the training#####################

#generate the input arrays
x_train_s = np.asarray(x_p_list['s'][:train_len], dtype='int')
x_train_l = np.asarray(x_p_list['l'][:train_len], dtype='int')
x_train_r = np.asarray(x_r_list[:train_len], dtype='int')
y_train = np.asarray(y_list[:train_len], dtype='int')

x_valid_s = np.asarray(x_p_list['s'][train_len:], dtype='int')
x_valid_l = np.asarray(x_p_list['l'][train_len:], dtype='int')
x_valid_r = np.asarray(x_r_list[train_len:], dtype='int')
y_valid = np.asarray(y_list[train_len:], dtype='int')

model.fit([x_train_s, x_train_l, x_train_r], y_train, 
          validation_data=([x_valid_s, x_valid_l, x_valid_r], y_valid),
          batch_size=batch_size, epochs=num_epoch)

# Save model and weights
add_h5 = model_name + '.h5'
save_dir = os.path.join(os.getcwd(), '../weight_bin')

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, add_h5)
model.save(model_path)
print('Save model')
del(model)

del(x_train_s, x_train_l, x_train_r, y_train)
del(x_valid_s, x_valid_l, x_valid_r, y_valid)

del(x_p_list, x_r_list, y_list)

generating big-batches 20000 400000
generating big-batches 40000 400000
generating big-batches 60000 400000
generating big-batches 80000 400000
generating big-batches 100000 400000
generating big-batches 120000 400000
generating big-batches 140000 400000
generating big-batches 160000 400000
generating big-batches 180000 400000
generating big-batches 200000 400000
generating big-batches 220000 400000
generating big-batches 240000 400000
generating big-batches 260000 400000
generating big-batches 280000 400000
generating big-batches 300000 400000
generating big-batches 320000 400000
generating big-batches 340000 400000
generating big-batches 360000 400000
generating big-batches 380000 400000
generating big-batches 400000 400000
Epoch 1/10
90000/90000 [==============================] - 1195s 13ms/step - loss: 0.4153 - binary_accuracy: 0.9012 - val_loss: 0.2901 - val_binary_accuracy: 0.9253
Epoch 2/10
90000/90000 [==============================] - 1194s 13ms/step - loss: 0.1424 - binary_ac

### Result on the testset for inductive link prediction

We use the testset for inductive link prediction.

In [111]:
data_name = 'WN18RR_v1'
model_id = 'main_4'

In [112]:
fine_tune = 'No'

In [113]:
#difine the names for saving
model_name = 'Model_' + model_id + '_' + data_name
ids_name = 'IDs_' + model_id + '_' + data_name

In [114]:
ids_name

'IDs_main_4_WN18RR_v1'

In [115]:
model_name

'Model_main_4_WN18RR_v1'

In [116]:
import librosa
import opensmile
import os
import sys
import numpy as np
import random
from collections import defaultdict
from copy import deepcopy
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model

In [117]:
class LoadKG:
    
    def __init__(self):
        
        self.x = 'Hello'
        
    def load_train_data(self, data_path, one_hop, data, s_t_r, entity2id, id2entity,
                     relation2id, id2relation):
        
        data_ = set()
    
        ####load the train, valid and test set##########
        with open (data_path, 'r') as f:
            
            data_ini = f.readlines()
                        
            for i in range(len(data_ini)):
            
                x = data_ini[i].split()
                
                x_ = tuple(x)
                
                data_.add(x_)
        
        ####relation dict#################
        index = len(relation2id)
     
        for key in data_:
            
            if key[1] not in relation2id:
                
                relation = key[1]
                
                relation2id[relation] = index
                
                id2relation[index] = relation
                
                index += 1
                
                #the inverse relation
                iv_r = '_inverse_' + relation
                
                relation2id[iv_r] = index
                
                id2relation[index] = iv_r
                
                index += 1
        
        #get the id of the inverse relation, by above definition, initial relation has 
        #always even id, while inverse relation has always odd id.
        def inverse_r(r):
            
            if r % 2 == 0: #initial relation
                
                iv_r = r + 1
            
            else: #inverse relation
                
                iv_r = r - 1
            
            return(iv_r)
        
        ####entity dict###################
        index = len(entity2id)
        
        for key in data_:
            
            source, target = key[0], key[2]
            
            if source not in entity2id:
                                
                entity2id[source] = index
                
                id2entity[index] = source
                
                index += 1
            
            if target not in entity2id:
                
                entity2id[target] = index
                
                id2entity[index] = target
                
                index += 1
                
        #create the set of triples using id instead of string        
        for ele in data_:
            
            s = entity2id[ele[0]]
            
            r = relation2id[ele[1]]
            
            t = entity2id[ele[2]]
            
            if (s,r,t) not in data:
                
                data.add((s,r,t))
            
            s_t_r[(s,t)].add(r)
            
            if s not in one_hop:
                
                one_hop[s] = dict()
            
            if r not in one_hop[s]:
                
                one_hop[s][r] = set()
            
            one_hop[s][r].add(t)
            
            if t not in one_hop:
                
                one_hop[t] = dict()
            
            r_inv = inverse_r(r)
            
            s_t_r[(t,s)].add(r_inv)
            
            if r_inv not in one_hop[t]:
                
                one_hop[t][r_inv] = set()
            
            one_hop[t][r_inv].add(s)

In [118]:
class ObtainPathsByDynamicProgramming:

    def __init__(self, size_bd=50, threshold=500000):
                
        self.size_bd = size_bd #size bound limit the number of paths to a target entity t
        
        #number of times paths with specific length been performed for recursion
        self.threshold = threshold
        
    #obtain all the connected entitis from one source s, by any paths
    #each connected target entity with their shortest path length will be recorded
    def obtain_all_connected_entity(self, s, one_hop):

        neighbor = {s: 0}

        for r in one_hop[s]:

            for t in one_hop[s][r]:

                if t not in neighbor:

                    neighbor[t] = 1

        completed = False
        length = 1

        while not completed:

            temp = set()

            for e in neighbor:

                for r in one_hop[e]:

                    for t in one_hop[e][r]:

                        if t not in neighbor:

                            temp.add(t)

            if len(temp) == 0:

                completed = True

            else:

                length += 1

                for t in temp:

                    neighbor[t] = length

        return(neighbor)
        
    '''
    Given an entity s, the function will find the paths from s to other entities, using recursion.
    
    One may refer to LeetCode Problem 797 for details:
        https://leetcode.com/problems/all-paths-from-source-to-target/
    '''
    def obtain_paths(self, mode, s, t_input, lower_bd, upper_bd, one_hop):

        if type(lower_bd) != type(1) or lower_bd < 1:
            
            raise TypeError("!!! invalid lower bound setting, must >= 1 !!!")
            
        if type(upper_bd) != type(1) or upper_bd < 1:
            
            raise TypeError("!!! invalid upper bound setting, must >= 1 !!!")
            
        if lower_bd > upper_bd:
            
            raise TypeError("!!! lower bound must not exced upper bound !!!")
            
        if s not in one_hop:
            
            raise ValueError('!!! entity not in one_hop. Please work on existing entities')
        
        #here is the result dict. Its key is each entity t sharing paths from s
        #The value of each t is a set containing the paths from s to t
        #These paths can be either the direct connection r, or a multi-hop path
        res = defaultdict(set)
        
        #qualified_t contains the types of t we want to consider,
        #that is, what t will be added to the result set.
        qualified_t = set()
        
        #under this mode, we will only consider the direct neighbour of s
        if mode == 'direct_neighbour':
            
            all_connected_to_t = dict()
        
            for r in one_hop[s]:
            
                for t in one_hop[s][r]:
                
                    qualified_t.add(t)
            
            #update the all_connected_to_t dict
            for t in qualified_t:

                temp_connected = self.obtain_all_connected_entity(t, one_hop)

                for key in temp_connected:

                    if key not in all_connected_to_t:

                        all_connected_to_t[key] = deepcopy(temp_connected[key])

                    else:

                        all_connected_to_t[key] = min(all_connected_to_t[key], temp_connected[key])
        
        #under this mode, we will only consider one specified entity t
        elif mode == 'target_specified':
            
            qualified_t.add(t_input)
            
            #all the connected entities from s to t
            all_connected_to_t = self.obtain_all_connected_entity(t_input, one_hop)
        
        #under this mode, we will consider any entity
        elif mode == 'any_target':
            
            #we want it to be empty
            all_connected_to_t = dict()
            
            for s_any in one_hop:
                
                qualified_t.add(s_any)
                
        else:
            
            raise ValueError('not a valid mode')
        
        '''
        We use recursion to find the paths
        On current node with the path [r1, ..., rk] and on-path entities {s, e1, ..., ek-1, node}
        from s to this node, we will further find the direct neighbor t' of this node. 
        If t' is not an on-path entity (not among s, e1,...ek-1, node), we recursively proceed to t' 
        '''
        def helper(node, path, on_path_en, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict):

            #when the current path is within lower_bd and upper_bd, 
            #and the node is among the qualified t, and it has not been fill of paths w.r.t size_limit,
            #we will add this path to the node
            if (len(path) >= lower_bd) and (len(path) <= upper_bd) and (
                node in qualified_t) and (len(res[node]) < self.size_bd):
                
                res[node].add(tuple(path))
                    
            #won't start new recursions if the current path length already reaches upper limit, or
            #the number of recursion on this length has reached the limit or
            if (len(path) < upper_bd) and (count_dict[len(path)] <= self.threshold):
         
                #find all the directly connected (r,t) pair to this current node
                #these pairs indicates potentially the next recursion
                potential_pair = set()

                for r in one_hop[node]:

                    for t in one_hop[node][r]:

                        potential_pair.add((r,t))

                '''here, we use the entities connected to t as priority'''
                #the more the entity close to t, the higher it will be ranked to recurse ahead
                priority_list = list()
                
                for Tuple in potential_pair:
                    
                    r, t = Tuple[0], Tuple[1]
                    
                    if t in all_connected_to_t:
                        
                        priority_list.append([Tuple, min(all_connected_to_t[t], upper_bd)])
                        
                    else:
                        
                        priority_list.append([Tuple, upper_bd])
                        
                priority_list.sort(key = lambda x: x[-1])
                
                for List in priority_list:
                    
                    r, t = List[0][0], List[0][1]
                    
                    count_dict[len(path)] += 1
                    
                    #if t not on the path, then finally proceed to next recursion
                    if t not in on_path_en:

                        helper(t, path + [r], on_path_en.union({t}), res, qualified_t, 
                               lower_bd, upper_bd, one_hop, count_dict)
        
        length_dict = defaultdict(int)
        count_dict = defaultdict(int)
        
        helper(s, [], {s}, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict)
        
        return(res, count_dict)

In [119]:
#load the classes
Class_1 = LoadKG()
Class_2 = ObtainPathsByDynamicProgramming()

In [120]:
#load ids and relation/entity dicts
with open('../weight_bin/' + ids_name + '.pickle', 'rb') as handle:
    Dict = pickle.load(handle)

one_hop = Dict['one_hop']
data = Dict['data']
s_t_r = Dict['s_t_r']
entity2id = Dict['entity2id']
id2entity = Dict['id2entity']
relation2id = Dict['relation2id']
id2relation = Dict['id2relation']

#we want to keep the initial entity/relation dicts
entity2id_ini = deepcopy(entity2id)
id2entity_ini = deepcopy(id2entity)
relation2id_ini = deepcopy(relation2id)
id2relation_ini = deepcopy(id2relation)

num_r = len(id2relation)
num_r

18

In [121]:
#load the model
model = keras.models.load_model('../weight_bin/' + model_name + '.h5')

In [122]:
ind_train_path = '../data/' + data_name + '_ind/train.txt'
ind_valid_path = '../data/' + data_name + '_ind/valid.txt'
ind_test_path = '../data/' + data_name + '_ind/test.txt'

In [123]:
#load the test dataset
one_hop_ind = dict() 
data_ind = set()
s_t_r_ind = defaultdict(set)

len_0 = len(relation2id)
size_0 = len(entity2id)

#fill in the sets and dicts
Class_1.load_train_data(ind_train_path, 
                        one_hop_ind, data_ind, s_t_r_ind,
                        entity2id, id2entity, relation2id, id2relation)

len_1 = len(relation2id)
size_1 = len(entity2id)

if len_0 != len_1:
    raise ValueError('unseen relation!')

In [124]:
print(size_0, size_1, len(data_ind))

2746 3668 1618


In [125]:
#load the test dataset
one_hop_test = dict() 
data_test = set()
s_t_r_test = defaultdict(set)

len_0 = len(relation2id)
size_0 = len(entity2id)

#fill in the sets and dicts
Class_1.load_train_data(ind_test_path, 
                        one_hop_test, data_test, s_t_r_test,
                        entity2id, id2entity, relation2id, id2relation)


len_1 = len(relation2id)
size_1 = len(entity2id)

if len_0 != len_1:
    raise ValueError('unseen relation!')

In [126]:
print(size_0, size_1, len(data_test))

3668 3668 188


In [127]:
#load the validation for existing triple removal when ranking
one_hop_valid = dict() 
data_valid = set()
s_t_r_valid = defaultdict(set)

len_0 = len(relation2id)
size_0 = len(entity2id)

#fill in the sets and dicts
Class_1.load_train_data(ind_valid_path, 
                        one_hop_valid, data_valid, s_t_r_valid,
                        entity2id, id2entity, relation2id, id2relation)

len_1 = len(relation2id)
size_1 = len(entity2id)

if len_0 != len_1:
    raise ValueError('unseen relation!')

In [128]:
print(size_0, size_1, len(data_valid))

3668 3668 185


In [129]:
print(len(entity2id), len(entity2id_ini))

3668 2746


In [130]:
valid_path = '../data/' + data_name + '/valid.txt'
test_path = '../data/' + data_name + '/test.txt'

In [131]:
#load the test dataset
one_hop_train_test = dict() 
data_train_test = set()
s_t_r_train_test = defaultdict(set)

len_0 = len(relation2id)
size_0 = len(entity2id)

#fill in the sets and dicts
Class_1.load_train_data(test_path, 
                        one_hop_train_test, data_train_test, s_t_r_train_test,
                        entity2id, id2entity, relation2id, id2relation)


len_1 = len(relation2id)
size_1 = len(entity2id)

if len_0 != len_1:
    raise ValueError('unseen relation!')

In [132]:
#load the validation for existing triple removal when ranking
one_hop_train_valid = dict() 
data_train_valid = set()
s_t_r_train_valid = defaultdict(set)

len_0 = len(relation2id)
size_0 = len(entity2id)

#fill in the sets and dicts
Class_1.load_train_data(valid_path, 
                        one_hop_train_valid, data_train_valid, s_t_r_train_valid,
                        entity2id, id2entity, relation2id, id2relation)

len_1 = len(relation2id)
size_1 = len(entity2id)

if len_0 != len_1:
    raise ValueError('unseen relation!')

In [133]:
#we want to check whether there are overlapping 
#between the entities of train triples and inductive test and valid triples
overlapping = 0

for ele in data_test:
    
    s, r, t = ele[0], ele[1], ele[2]
    
    if s in id2entity_ini or t in id2entity_ini:
        
        overlapping += 1
        
overlapping

0

In [134]:
overlapping = 0

for ele in data_valid:
    
    s, r, t = ele[0], ele[1], ele[2]
    
    if s in id2entity_ini or t in id2entity_ini:
        
        overlapping += 1
        
overlapping

0

In [135]:
#we want to check whether there are overlapping 
#between the entities of train triples and inductive test and valid triples
overlapping = 0

for ele in data_ind:
    
    s, r, t = ele[0], ele[1], ele[2]
    
    if s in id2entity_ini or t in id2entity_ini:
        
        overlapping += 1
        
overlapping

0

**Fine tune if necessary**

In [60]:
#function to build all the big batches
def build_big_batches(holder_len, lower_bd, upper_bd, Class_2, one_hop, s_t_r,
                      x_p_list, x_r_list, y_list,
                      relation2id, entity2id, id2relation, id2entity):
    
    if holder_len % 10 != 0:
        raise ValueError('We would like to take 10X as a big-batch size')
    
    #the set of all relation IDs
    relation_id_set = set()
    for i in range(len(id2relation)):
        
        if i not in id2relation:
            raise ValueError('error when generaing id2relation')
        
        relation_id_set.add(i)
    
    num_r = len(id2relation)
    
    #count how many appending has performed
    count = 0

    #in case not all entities in entity2id are in one_hop, 
    #so we need to find out who are indeed in
    existing_ids = set()
    
    for s_1 in one_hop:
        existing_ids.add(s_1)
        
    existing_ids = list(existing_ids)
    
    carry_on = True
    
    while carry_on:

        #obtain paths by dynamic programming
        source_id = random.choice(existing_ids)

        result, count_dict = Class_2.obtain_paths('direct_neighbour', source_id, 
                                                   'not_specified', lower_bd, upper_bd, one_hop)
        
        #We want to increase the diversity of paths and targets.
        #So we abandon one sub-graph from a source_id, if we sampled more than K1 path pairs
        #Note that we mean "sampled", not "appended"! 
        #We do not care whether the pair is actually appended.
        threshold_0 = 1000
        count_0 = 0
        
        for target_id in result:

            if (not carry_on) or (count_0 > threshold_0):
                break
            
            #we want to make sure s, t are indeed directly connected, 
            #otherwise there is no relation for positive sample
            #also, we want to make sure s and t and not connected by all relations, 
            #although this situation is rare. 
            #But in that case, there is no relation for negative samples
            #Also, we want at least two different paths here between s and t
            if ((source_id, target_id) in s_t_r) and (
                len(s_t_r[(source_id, target_id)]) < len(id2relation)) and (
                len(result[target_id]) >= 2):
                
                dir_r = list(s_t_r[(source_id, target_id)])
                
                non_dir_r = list(relation_id_set.difference(dir_r))
                
                if len(dir_r) <= 0:
                    
                    raise ValueError('errors when creating s_t_r !!')
                    
                temp_path_list = list(result[target_id])
                    
                #futhermore, we will abandon one targed_id if we sampled more than K2 times
                threshold_1 = 50
                count_1 = 0
                
                while count_1 <= threshold_1 and count_0 <= threshold_0:
                
                    temp_pair = random.sample(temp_path_list,2)
                    
                    path_1, path_2 = temp_pair[0], temp_pair[1]

                    #decide which path is shorter and which is longer
                    if len(path_1) <= len(path_2):

                        path_s, path_l = path_1, path_2

                    else:

                        path_s, path_l = path_2, path_1                            

                    if (len(path_s) < lower_bd) or (len(path_l) > upper_bd):

                        raise ValueError('something wrong with the path finding')

                    #proceed when the entire length not yet reached,
                    #and whether this path pair is new, and whether the two paths are different
                    #But it is optional to require the path to be new. 
                    #We may remove this requirment, especially for short paths
                    '''remember to cancel the comment below when using path_comb'''
                    if (carry_on) and (path_s != path_l):

                        #we always add one positive and one negative situation together,
                        #hence, the length of list should always be even.
                        #also we want to make sure the length of lists coincide
                        if (len(x_p_list['s']) != len(y_list)) or (
                            len(x_p_list['s']) != len(x_p_list['l'])) or (
                            len(y_list) != len(x_r_list)) or (
                            len(y_list) % 2 != 0):

                            raise ValueError('error when building big batches: length error')

                        #####positive#####################
                        #we randomly choose one direction relation as the target relation
                        relation_id = random.choice(dir_r)

                        #append the paths: note that we add the space holder id at the end
                        #of the shorter path
                        x_p_list['s'].append(list(path_s) + [num_r]*abs(len(path_s)-upper_bd))
                        x_p_list['l'].append(list(path_l) + [num_r]*abs(len(path_l)-upper_bd))

                        #append relation
                        x_r_list.append([relation_id])
                        y_list.append(1.)

                        #####negative#####################
                        relation_id = random.choice(non_dir_r)

                        #append the paths: note that we add the space holder id at the end
                        #of the shorter path
                        x_p_list['s'].append(list(path_s) + [num_r]*abs(len(path_s)-upper_bd))
                        x_p_list['l'].append(list(path_l) + [num_r]*abs(len(path_l)-upper_bd))

                        #append relation
                        x_r_list.append([relation_id])
                        y_list.append(0.)

                        ######add to path combinations#####
                        #here is the tricky part: we have to add both (path_s, path_l)
                        #and (path_l, path_s). This is because when the length are the same
                        #adding only one situation won't guarantee that 
                        #the same path with different order is also considered.
                        #in other words: path combination don't have order, but our dict does.
                        #so we have to add both situations.
                        '''remember to cancel the comment here when using path_comb'''
                        #path_comb[(len(path_s), len(path_l))].add((path_s, path_l))
                        #path_comb[(len(path_s), len(path_l))].add((path_l, path_s))

                        count += 2

                        if count % 2000 == 0:
                            print('generating big-batches', count, holder_len)

                    if len(y_list) >= holder_len:

                        carry_on = False
                        
                    count_1 += 1
                    count_0 += 1

In [27]:
#define the lists
x_p_list, x_r_list, y_list = {'s': [], 'l': []}, list(), list()

#######################################
###build the big-batches###############      

#fill in the training array list
build_big_batches(20000, 2, 10, Class_2, one_hop_ind, s_t_r_ind,
                      x_p_list, x_r_list, y_list,
                      relation2id, entity2id, id2relation, id2entity)

x_test_s = np.asarray(x_p_list['s'], dtype='int')
x_test_l = np.asarray(x_p_list['l'], dtype='int')
x_test_r = np.asarray(x_r_list, dtype='int')
y_test = np.asarray(y_list, dtype='int')

if fine_tune == 'Yes':

    model.fit([x_test_s, x_test_l, x_test_r], y_test, batch_size=4, epochs=2)
    
else:
    
    model.evaluate([x_test_s, x_test_l, x_test_r], y_test, batch_size=4)

generating big-batches 2000 20000
generating big-batches 4000 20000
generating big-batches 6000 20000
generating big-batches 8000 20000
generating big-batches 10000 20000
generating big-batches 12000 20000
generating big-batches 14000 20000
generating big-batches 16000 20000
generating big-batches 18000 20000
generating big-batches 20000 20000
5000/5000 [==============================] - 22s 4ms/step - loss: 0.5888 - binary_accuracy: 0.8870


In [57]:
def relation_ranking(s, t, lower_bd, upper_bd, one_hop, id2relation, model):
    
    path_holder = set()
    
    for iteration in range(20):
    
        result, count_dict = Class_2.obtain_paths('target_specified', s, t, lower_bd, upper_bd, one_hop)
        if t in result:
            
            for path in result[t]:
                
                path_holder.add(path)
    
    path_holder = list(path_holder)
    random.shuffle(path_holder)
    
    score_dict = defaultdict(float)
    
    count = 0
    
    if len(path_holder) >= 2:
    
        #iterate over path_1
        while count <= 50:

            temp_pair = random.sample(path_holder, 2)

            path_1, path_2 = temp_pair[0], temp_pair[1]

            #decide which path is shorter and which is longer
            if len(path_1) <= len(path_2):

                path_s, path_l = path_1, path_2

            else:

                path_s, path_l = path_2, path_1                            

            #whether lengths of the two paths satisfies the requirments
            if (len(path_s) < lower_bd) or (len(path_l) > upper_bd):

                raise ValueError('something wrong with path finding')

            list_s = list()
            list_l = list()
            list_r = list()

            for i in range(len(id2relation)):

                if i not in id2relation:

                    raise ValueError ('error when generating id2relation')

                list_s.append(list(path_s) + [num_r]*abs(len(path_s)-upper_bd))
                list_l.append(list(path_l) + [num_r]*abs(len(path_l)-upper_bd))
                list_r.append([i])

            input_s = np.array(list_s)
            input_l = np.array(list_l)
            input_r = np.array(list_r)

            pred = model.predict([input_s, input_l, input_r], verbose = 0)

            for i in range(pred.shape[0]):

                score_dict[i] += float(pred[i])

            count += 1
    
    count_abv_1 = 0
    
    for key in score_dict:
        
        if score_dict[key] >= 0.8:
            
            count_abv_1 += 1
    
    print(len(score_dict), count_abv_1, len(path_holder))

    return(score_dict)

In [29]:
########################################################
#obtain the precision-recall area under curve (AUC-PR)##

#select 500 triples randomly, unless no such many
selected = random.sample(list(data_test), min(len(data_test), 500))

random.shuffle(selected)

###Hit at 1#############################
#generate the negative samples by randomly replace relation with all the other relaiton
Hits_at_1 = 0
Hits_at_3 = 0
Hits_at_10 = 0
MRR_raw = 0.

for i in range(len(selected)):
    
    s_true, r_true, t_true = selected[i][0], selected[i][1], selected[i][2]
    
    score_dict = relation_ranking(s_true, t_true, 2, 10, one_hop_ind, id2relation, model)
    
    #[... [score, r], ...]
    temp_list = list()
    
    for r in id2relation:
        
        if r in score_dict:
            
            temp_list.append([score_dict[r], r])
            
        else:
            
            temp_list.append([0.0, r])
        
    sorted_list = sorted(temp_list, key = lambda x: x[0], reverse=True)
    
    p = 0
    exist_tri = 0
    
    while p < len(sorted_list) and sorted_list[p][1] != r_true:
        
        #moreover, we want to remove existing triples
        if ((s_true, sorted_list[p][1], t_true) in data_test) or (
            (s_true, sorted_list[p][1], t_true) in data_valid) or (
            (s_true, sorted_list[p][1], t_true) in data_ind) or (
            (s_true, sorted_list[p][1], t_true) in data) or (
            (s_true, sorted_list[p][1], t_true) in data_train_valid) or (
            (s_true, sorted_list[p][1], t_true) in data_train_test):
            
            exist_tri += 1
            
        p += 1
    
    if p - exist_tri == 0:
        
        Hits_at_1 += 1
        
    if p - exist_tri < 3:
        
        Hits_at_3 += 1
        
    if p - exist_tri < 10:
        
        Hits_at_10 += 1
        
    MRR_raw += 1./float(p - exist_tri + 1.) 
        
    print('checkcorrect', r_true, sorted_list[p][1],
          'real score', sorted_list[p][0],
          'Hits@1', Hits_at_1/(i+1),
          'Hits@3', Hits_at_3/(i+1),
          'Hits@10', Hits_at_10/(i+1),
          'MRR', MRR_raw/(i+1),
          'cur_rank', p - exist_tri,
          'abs_cur_rank', p,
          'total_num', i, len(selected))

18 2 91
checkcorrect 2 2 real score 50.23208004236221 Hits@1 1.0 Hits@3 1.0 Hits@10 1.0 MRR 1.0 cur_rank 0 abs_cur_rank 0 total_num 0 188
0 0 0
checkcorrect 2 2 real score 0.0 Hits@1 0.5 Hits@3 1.0 Hits@10 1.0 MRR 0.6666666666666666 cur_rank 2 abs_cur_rank 2 total_num 1 188
0 0 0
checkcorrect 4 4 real score 0.0 Hits@1 0.3333333333333333 Hits@3 0.6666666666666666 Hits@10 1.0 MRR 0.5111111111111111 cur_rank 4 abs_cur_rank 4 total_num 2 188
18 2 544
checkcorrect 2 2 real score 48.240289747714996 Hits@1 0.25 Hits@3 0.75 Hits@10 1.0 MRR 0.5083333333333333 cur_rank 1 abs_cur_rank 1 total_num 3 188
18 2 1390
checkcorrect 2 2 real score 49.834606766700745 Hits@1 0.2 Hits@3 0.8 Hits@10 1.0 MRR 0.5066666666666666 cur_rank 1 abs_cur_rank 1 total_num 4 188
0 0 0
checkcorrect 2 2 real score 0.0 Hits@1 0.16666666666666666 Hits@3 0.8333333333333334 Hits@10 1.0 MRR 0.4777777777777778 cur_rank 2 abs_cur_rank 2 total_num 5 188
18 2 8
checkcorrect 2 2 real score 48.945710241794586 Hits@1 0.14285714285714

18 4 1112
checkcorrect 4 4 real score 42.00411120057106 Hits@1 0.2708333333333333 Hits@3 0.875 Hits@10 0.9583333333333334 MRR 0.5370054713804712 cur_rank 0 abs_cur_rank 0 total_num 47 188
18 3 74
checkcorrect 4 4 real score 40.076682925224304 Hits@1 0.2857142857142857 Hits@3 0.8775510204081632 Hits@10 0.9591836734693877 MRR 0.546454339311482 cur_rank 0 abs_cur_rank 0 total_num 48 188
0 0 0
checkcorrect 10 10 real score 0.0 Hits@1 0.28 Hits@3 0.86 Hits@10 0.94 MRR 0.5373434343434341 cur_rank 10 abs_cur_rank 10 total_num 49 188
18 2 6
checkcorrect 4 4 real score 50.81388485431671 Hits@1 0.29411764705882354 Hits@3 0.8627450980392157 Hits@10 0.9411764705882353 MRR 0.5464151317092492 cur_rank 0 abs_cur_rank 0 total_num 50 188
0 0 0
checkcorrect 2 2 real score 0.0 Hits@1 0.28846153846153844 Hits@3 0.8653846153846154 Hits@10 0.9423076923076923 MRR 0.5423174048174046 cur_rank 2 abs_cur_rank 2 total_num 51 188
18 2 40
checkcorrect 2 2 real score 50.09546935558319 Hits@1 0.2830188679245283 Hits@

0 0 0
checkcorrect 4 4 real score 0.0 Hits@1 0.2717391304347826 Hits@3 0.8804347826086957 Hits@10 0.9565217391304348 MRR 0.5437472551602988 cur_rank 4 abs_cur_rank 4 total_num 91 188
18 2 56
checkcorrect 2 2 real score 48.92614138126373 Hits@1 0.26881720430107525 Hits@3 0.8817204301075269 Hits@10 0.956989247311828 MRR 0.5432768545671772 cur_rank 1 abs_cur_rank 1 total_num 92 188
18 4 2
checkcorrect 2 2 real score -4.996071130037308 Hits@1 0.26595744680851063 Hits@3 0.8723404255319149 Hits@10 0.9468085106382979 MRR 0.5383156441667082 cur_rank 12 abs_cur_rank 12 total_num 93 188
0 0 0
checkcorrect 2 2 real score 0.0 Hits@1 0.2631578947368421 Hits@3 0.8736842105263158 Hits@10 0.9473684210526315 MRR 0.5361579356316201 cur_rank 2 abs_cur_rank 2 total_num 94 188
0 0 0
checkcorrect 2 2 real score 0.0 Hits@1 0.2604166666666667 Hits@3 0.875 Hits@10 0.9479166666666666 MRR 0.5340451793576796 cur_rank 2 abs_cur_rank 2 total_num 95 188
18 2 72
checkcorrect 2 2 real score 50.38349676132202 Hits@1 0.

0 0 0
checkcorrect 2 2 real score 0.0 Hits@1 0.3037037037037037 Hits@3 0.8814814814814815 Hits@10 0.9481481481481482 MRR 0.5590333534777981 cur_rank 2 abs_cur_rank 2 total_num 134 188
18 2 70
checkcorrect 2 2 real score 50.34054511785507 Hits@1 0.3014705882352941 Hits@3 0.8823529411764706 Hits@10 0.9485294117647058 MRR 0.5585992847022262 cur_rank 1 abs_cur_rank 1 total_num 135 188
18 3 12
checkcorrect 2 2 real score 44.425992518663406 Hits@1 0.29927007299270075 Hits@3 0.8832116788321168 Hits@10 0.948905109489051 MRR 0.5581715526971004 cur_rank 1 abs_cur_rank 1 total_num 136 188
0 0 1
checkcorrect 2 2 real score 0.0 Hits@1 0.2971014492753623 Hits@3 0.8840579710144928 Hits@10 0.9492753623188406 MRR 0.5565422902379427 cur_rank 2 abs_cur_rank 2 total_num 137 188
18 2 29
checkcorrect 2 2 real score 50.55068492889404 Hits@1 0.302158273381295 Hits@3 0.8848920863309353 Hits@10 0.9496402877697842 MRR 0.5597326334736409 cur_rank 0 abs_cur_rank 0 total_num 138 188
18 2 6
checkcorrect 2 2 real sco

18 2 10
checkcorrect 2 2 real score 49.437060832977295 Hits@1 0.28651685393258425 Hits@3 0.8707865168539326 Hits@10 0.9550561797752809 MRR 0.5470622386352723 cur_rank 1 abs_cur_rank 1 total_num 177 188
0 0 0
checkcorrect 2 2 real score 0.0 Hits@1 0.2849162011173184 Hits@3 0.8715083798882681 Hits@10 0.9553072625698324 MRR 0.5458682224045351 cur_rank 2 abs_cur_rank 2 total_num 178 188
18 6 11
checkcorrect 2 2 real score 23.070589824346825 Hits@1 0.2833333333333333 Hits@3 0.8722222222222222 Hits@10 0.9555555555555556 MRR 0.5456133989467321 cur_rank 1 abs_cur_rank 1 total_num 179 188
18 2 176
checkcorrect 2 2 real score 50.305962562561035 Hits@1 0.287292817679558 Hits@3 0.8729281767955801 Hits@10 0.9558011049723757 MRR 0.5481238221569712 cur_rank 0 abs_cur_rank 0 total_num 180 188
0 0 0
checkcorrect 2 2 real score 0.0 Hits@1 0.2857142857142857 Hits@3 0.8736263736263736 Hits@10 0.9560439560439561 MRR 0.5469436546359622 cur_rank 2 abs_cur_rank 2 total_num 181 188
0 0 0
checkcorrect 2 2 real 

**Bin**

In [136]:
one_hop_ind

{2746: {4: {2747}, 2: {3106, 3122, 3406}, 3: {3106, 3122}},
 2747: {5: {2746}},
 2748: {2: {2749}, 3: {2749, 3281}, 4: {3607}},
 2749: {3: {2748, 2755, 3280, 3292, 3385}, 2: {2748, 3292, 3385}, 4: {3581}},
 2750: {2: {2751, 2942, 3254, 3451}, 3: {2751, 2942, 3254}, 4: {3429}},
 2751: {3: {2750}, 2: {2750, 3116}, 4: {3351}, 5: {3394}},
 2752: {2: {2753, 3515}, 3: {2753, 2765, 3535}, 5: {2817, 3232}},
 2753: {3: {2752, 3265, 3333}, 2: {2752, 3265, 3333}, 4: {2922}},
 2754: {2: {2755}, 3: {2755}},
 2755: {3: {2754, 3160, 3304}, 2: {2749, 2754, 3160, 3193, 3304}},
 2756: {2: {2757, 3115}, 5: {3163}, 4: {3315}, 3: {2757}},
 2757: {3: {2756, 2854}, 4: {3478}, 2: {2756}},
 2758: {2: {2759, 2914, 2986, 3069, 3632},
  4: {3578},
  3: {2759, 2914, 2986, 3632}},
 2759: {3: {2758}, 5: {3264}, 2: {2758}},
 2760: {2: {2761}, 3: {2761}},
 2761: {3: {2760, 2813}, 2: {2760, 2813}},
 2762: {2: {2763}, 3: {2763}, 4: {2956}},
 2763: {3: {2762, 2814, 2849}, 2: {2762, 2814}},
 2764: {4: {2765}, 3: {3232}, 2

In [137]:
id2relation

{0: '_has_part',
 1: '_inverse__has_part',
 2: '_derivationally_related_form',
 3: '_inverse__derivationally_related_form',
 4: '_hypernym',
 5: '_inverse__hypernym',
 6: '_verb_group',
 7: '_inverse__verb_group',
 8: '_synset_domain_topic_of',
 9: '_inverse__synset_domain_topic_of',
 10: '_also_see',
 11: '_inverse__also_see',
 12: '_similar_to',
 13: '_inverse__similar_to',
 14: '_member_meronym',
 15: '_inverse__member_meronym',
 16: '_instance_hypernym',
 17: '_inverse__instance_hypernym'}

In [165]:
result, count_dict = Class_2.obtain_paths('target_specified', 
                                          3163, 3007, 1, 880, one_hop_ind)

In [166]:
result[3007]

{(3,)}

In [168]:
count_dict

defaultdict(int,
            {0: 2,
             1: 13,
             2: 49,
             3: 138,
             4: 367,
             5: 666,
             6: 1385,
             7: 3507,
             8: 11404,
             9: 42544,
             10: 135169,
             11: 424550,
             12: 500001,
             13: 500003,
             14: 500002,
             15: 500003,
             16: 500004,
             17: 500006,
             18: 500001,
             19: 500014,
             20: 500003,
             21: 500005,
             22: 500006,
             23: 500004,
             24: 500004,
             25: 500002,
             26: 500001,
             27: 500006,
             28: 500002,
             29: 500001,
             30: 500001,
             31: 500005,
             32: 500004,
             33: 500003,
             34: 500003,
             35: 500004,
             36: 500005,
             37: 500003,
             38: 500006,
             39: 500001,
             40: 5000

In [142]:
def obtain_all_connected_entity(s, one_hop):

    neighbor = {s: 0}

    for r in one_hop[s]:

        for t in one_hop[s][r]:

            if t not in neighbor:

                neighbor[t] = 1

    completed = False
    length = 1

    while not completed:

        temp = set()

        for e in neighbor:

            for r in one_hop[e]:

                for t in one_hop[e][r]:

                    if t not in neighbor:

                        temp.add(t)

        if len(temp) == 0:

            completed = True

        else:

            length += 1

            for t in temp:

                neighbor[t] = length

    return(neighbor)

In [143]:
#given an entity s, we want to obtain all its connected entities not through t 
def exclusive_connection(s, t_input, one_hop):

    neighbor = {s: 0}

    for r in one_hop[s]:

        for t in one_hop[s][r]:

            if (t != t_input) and (t not in neighbor):

                neighbor[t] = 1

    completed = False
    length = 1

    while not completed:

        temp = set()

        for e in neighbor:

            for r in one_hop[e]:

                for t in one_hop[e][r]:

                    if (t != t_input) and (t not in neighbor):

                        temp.add(t)

        if len(temp) == 0:

            completed = True

        else:

            length += 1

            for t in temp:

                neighbor[t] = length

    return(neighbor)

In [144]:
#obtain the connected entites by our algorithm
def connection_by_our_alrogithm(s, t, lower_bd, upper_bd, one_hop, id2relation):
    
    path_holder = set()
    
    for iteration in range(4):
    
        result, count_dict = Class_2.obtain_paths('target_specified', s, t, lower_bd, upper_bd, one_hop)
        
        if t in result:
            
            for path in result[t]:
                
                path_holder.add(path)
    
    path_holder = list(path_holder)
    
    return(path_holder)

In [147]:
selected = random.sample(list(data_test), min(len(data_test), 500))

#random.shuffle(selected)

for i in range(len(selected)):
    
    s_true, r_true, t_true = selected[i][0], selected[i][1], selected[i][2]
    
    path_holder = connection_by_our_alrogithm(s_true, t_true, 1, 10, one_hop_ind, id2relation)
            
    neighbor_s = exclusive_connection(s_true, t_true, one_hop_ind)

    set_s = set()

    for key in neighbor_s:

        set_s.add(key)

    neighbor_t = exclusive_connection(t_true, s_true, one_hop_ind)

    set_t = set()

    for key in neighbor_t:

        set_t.add(key)
        
    if len(path_holder) <= 1:
        
        all_neighbor_s = obtain_all_connected_entity(s_true, one_hop_ind)
        
        if t_true in all_neighbor_s:
            
            dist = all_neighbor_s[t_true]
            
        else:
            
            dist = 'not reachable'
        
        print(i, s_true, t_true, len( set_s.intersection(set_t) ), len(path_holder), dist)


4 3426 3481 0 1 1
5 2847 3339 0 1 1
8 3195 3028 0 1 1
9 3251 3126 0 1 1
11 3001 3654 0 1 1
13 3095 3043 0 0 not reachable
14 2807 3605 0 1 1
15 3499 3614 0 0 not reachable
18 3048 3039 868 1 1
20 3398 3080 0 1 1
22 3557 3223 867 0 13
23 3647 3305 868 0 14
26 3163 3007 868 1 1
28 3425 3662 863 0 12
42 3365 3364 0 1 1
44 2752 3535 0 1 1
46 3052 2847 0 0 not reachable
47 3420 3001 0 1 1
48 3515 2752 0 1 1
52 2923 3615 0 1 1
53 2985 3121 0 1 1
58 3088 3177 865 0 20
60 2749 2755 0 1 1
62 3138 3630 0 1 1
67 3253 3628 0 0 not reachable
70 3658 2807 0 1 1
71 3500 3654 0 0 not reachable
72 3087 3583 0 1 1
80 3510 3111 0 0 not reachable
93 3509 3030 0 1 1
94 3627 3248 0 0 not reachable
95 2775 2774 0 1 1
98 3363 3118 0 1 1
101 2966 2886 868 1 2
102 3572 3150 0 1 1
103 3210 3212 0 1 1
104 3234 3435 0 0 not reachable
108 2812 3343 0 1 1
109 3373 3356 0 1 1
110 2915 3641 0 1 1
116 3311 3659 0 0 not reachable
120 3100 3269 868 1 1
121 2781 3415 864 1 1
122 3550 3226 0 0 not reachable
123 3111 3510 0

In [ ]:
class ObtainPathsByDynamicProgramming:

    def __init__(self, size_bd=50, threshold=5000):
                
        self.size_bd = size_bd #size bound limit the number of paths to a target entity t
        
        #number of times paths with specific length are performed
        self.threshold = threshold
        
    #obtain all the connected entitis from one source s, by any paths
    #each connected target entity with their shortest path length will be recorded
    def obtain_all_connected_entity(self, s, one_hop):

        neighbor = {s: 0}

        for r in one_hop[s]:

            for t in one_hop[s][r]:

                if t not in neighbor:

                    neighbor[t] = 1

        completed = False
        length = 1

        while not completed:

            temp = set()

            for e in neighbor:

                for r in one_hop[e]:

                    for t in one_hop[e][r]:

                        if t not in neighbor:

                            temp.add(t)

            if len(temp) == 0:

                completed = True

            else:

                length += 1

                for t in temp:

                    neighbor[t] = length

        return(neighbor)
        
    '''
    Given an entity s, the function will find the paths from s to other entities, using recursion.
    
    One may refer to LeetCode Problem 797 for details:
        https://leetcode.com/problems/all-paths-from-source-to-target/
    '''
    def obtain_paths(self, mode, s, t_input, lower_bd, upper_bd, one_hop):

        if type(lower_bd) != type(1) or lower_bd < 1:
            
            raise TypeError("!!! invalid lower bound setting, must >= 1 !!!")
            
        if type(upper_bd) != type(1) or upper_bd < 1:
            
            raise TypeError("!!! invalid upper bound setting, must >= 1 !!!")
            
        if lower_bd > upper_bd:
            
            raise TypeError("!!! lower bound must not exced upper bound !!!")
            
        if s not in one_hop:
            
            raise ValueError('!!! entity not in one_hop. Please work on existing entities')
        
        #here is the result dict. Its key is each entity t sharing paths from s
        #The value of each t is a set containing the paths from s to t
        #These paths can be either the direct connection r, or a multi-hop path
        res = defaultdict(set)
        
        #qualified_t contains the types of t we want to consider,
        #that is, what t will be added to the result set.
        qualified_t = set()
        
        #all the connected entities from s, with any path any length
        all_connected = self.obtain_all_connected_entity(s, one_hop)
        
        #under this mode, we will only consider the direct neighbour of s
        if mode == 'direct_neighbour':
        
            for r in one_hop[s]:
            
                for t in one_hop[s][r]:
                
                    qualified_t.add(t)
        
        #under this mode, we will only consider one specified entity t
        elif mode == 'target_specified':
            
            qualified_t.add(t_input)
        
        #under this mode, we will consider any entity
        elif mode == 'any_target':
            
            for s_any in all_connected:
                
                qualified_t.add(s_any)
                
        else:
            
            raise ValueError('not a valid mode')
        
        '''
        We use recursion to find the paths
        On current node with the path [r1, ..., rk] and on-path entities {s, e1, ..., ek-1, node}
        from s to this node, we will further find the direct neighbor t' of this node. 
        If t' is not an on-path entity (not among s, e1,...ek-1, node), we recursively proceed to t' 
        '''
        def helper(node, path, on_path_en, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict):

            #when the current path is within lower_bd and upper_bd, 
            #and the node is among the qualified t, we will then intend to add this node 
            if (len(path) >= lower_bd) and (len(path) <= upper_bd) and (node in qualified_t):
                
                #Further more: the node is new in the result set, 
                #or its detected path not yet reach the threshold, we will finally add this path
                if (node not in res) or (len(res[node]) < self.size_bd):
                
                    res[node].add(tuple(path))
                    
            #won't start new recursions if the current path length already reaches upper limit, or
            if len(path) < upper_bd:
         
                #find all the directly connected (r,t) pair to this current node
                #these pairs indicates potentially the next recursion
                potential_pair = set()

                for r in one_hop[node]:

                    for t in one_hop[node][r]:

                        potential_pair.add((r,t))

                potential_pair_list = list(potential_pair)
                random.shuffle(potential_pair_list)

                for Tuple in potential_pair_list:
                    
                    r, t = Tuple[0], Tuple[1]
                    
                    #if t already on the path, we won't bother to check further
                    if t not in on_path_en:
                        
                        #We start the new recursion, if the next entity t has not been reached
                        #by paths of the same length for the limited times.
                        proceed = False
                        if len(path)+1 not in count_dict[t]:

                            count_dict[t][len(path)+1] = 1
                            proceed = True

                        elif count_dict[t][len(path)+1] < self.threshold:

                            count_dict[t][len(path)+1] += 1
                            proceed = True

                        #but of course if t already on the path, we won't proceed.
                        if proceed:

                            helper(t, path + [r], on_path_en.union({t}), res, qualified_t, 
                               lower_bd, upper_bd, one_hop, count_dict)
        
        length_dict = defaultdict(int)
        count_dict = defaultdict(dict)
        
        helper(s, [], {s}, res, qualified_t, lower_bd, upper_bd, one_hop, count_dict)
        
        return(res, count_dict)

In [118]:
test_1 = {1: 2, 2: 5}
test_2 = {1: 3, 0: 4}
test_1.update(test_2)

In [119]:
test_1

{1: 3, 2: 5, 0: 4}

In [120]:
test_2

{1: 3, 0: 4}

In [127]:
priority_list = [[(1,2), 5], [(3,4), 2], [(5,6), 3]]

priority_list.sort(key = lambda x: x[-1], reverse=True)

priority_list

[[(1, 2), 5], [(5, 6), 3], [(3, 4), 2]]